In [13]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as FF
import plotly
plotly.tools.set_credentials_file(username='ninjamouse', api_key='LejqbK7NWhPU7zXoMuhw')

import numpy as np
import pandas as pd

df = pd.read_csv('first_attack.csv')

In [14]:
df.head()

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,"2010-06-12 06:50:00,055616",88.208.248.162,192.168.4.120,HTTP,1514,Continuation
1,2,"2010-06-12 06:50:00,059867",88.208.248.162,192.168.4.120,HTTP,1514,Continuation
2,3,"2010-06-12 06:50:00,061151",192.168.4.120,88.208.248.162,TCP,60,2677 > 80 [ACK] Seq=1 Ack=2921 Win=17520 Len=0
3,4,"2010-06-12 06:50:00,062786",208.122.28.2,192.168.4.120,HTTP,1195,HTTP/1.1 200 OK (JPEG JFIF image)
4,5,"2010-06-12 06:50:00,064234",192.168.4.120,208.122.28.10,HTTP,140,GET /profile_images/159516/rollinsliar_small.j...


In [15]:
df.Source.value_counts()

192.168.4.120            15032
74.63.40.21               5735
192.168.56.1              4733
75.127.97.72              4218
174.133.126.130           3747
192.168.5.122             2868
67.220.214.50             2471
192.168.2.110             2204
88.208.248.162            1590
207.210.64.84             1236
208.122.28.3               922
202.41.220.218             623
72.21.211.163              590
192.168.4.121              566
208.122.28.2               566
208.73.210.28              501
66.246.216.94              451
208.94.3.104               450
68.142.205.139             444
208.122.28.4               392
192.168.2.113              307
208.122.28.10              301
66.220.146.18              281
192.168.1.104              271
69.192.24.88               266
68.142.205.137             263
208.122.28.20              263
208.122.28.30              261
192.168.2.108              259
208.122.28.18              250
                         ...  
69.147.125.65                4
Ibm_bb:c

In [16]:
df.Destination.value_counts()

192.168.4.120        23440
75.127.97.72          4719
192.168.56.1          4218
74.63.40.21           3170
192.168.2.110         2767
174.133.126.130       2272
67.220.214.50         2001
192.168.5.122         1354
88.208.248.162         880
207.210.64.84          878
192.168.4.121          716
208.122.28.3           705
208.122.28.2           469
192.168.2.113          439
202.41.220.218         420
192.168.2.108          413
208.122.28.4           353
66.246.216.94          332
72.21.211.163          302
208.73.210.28          300
192.168.1.103          271
208.122.28.10          259
192.168.1.101          254
192.168.1.104          249
208.122.28.29          243
198.164.30.2           243
68.142.205.139         239
208.122.28.30          238
208.94.3.104           229
208.122.28.20          225
                     ...  
65.54.81.218             6
64.202.189.170           5
209.80.46.80             5
65.55.15.125             5
209.80.46.53             5
209.80.46.36             5
2

In [17]:
dest=df[df.Destination=='75.127.97.72']
dest

,No.,Time,Source,Destination,Protocol,Length,Info
31173,31174,"2010-06-12 06:54:06,347684",192.168.56.1,75.127.97.72,TCP,74,52998 > 80 [SYN] Seq=0 Win=29200 Len=0 MSS=1...
31175,31176,"2010-06-12 06:54:06,348033",192.168.56.1,75.127.97.72,TCP,66,52998 > 80 [ACK] Seq=1 Ack=1 Win=29696 Len=0...
31176,31177,"2010-06-12 06:54:06,348044",192.168.56.1,75.127.97.72,HTTP,710,GET /drupal7/ HTTP/1.1
31184,31185,"2010-06-12 06:54:06,552973",192.168.56.1,75.127.97.72,TCP,66,52998 > 80 [ACK] Seq=645 Ack=4345 Win=37888 ...
31186,31187,"2010-06-12 06:54:06,553056",192.168.56.1,75.127.97.72,TCP,66,52998 > 80 [ACK] Seq=645 Ack=4444 Win=37888 ...
31192,31193,"2010-06-12 06:54:06,585868",192.168.56.1,75.127.97.72,HTTP,749,GET /drupal7/sites/default/files/styles/medium...
31194,31195,"2010-06-12 06:54:06,586143",192.168.56.1,75.127.97.72,TCP,74,52999 > 80 [SYN] Seq=0 Win=29200 Len=0 MSS=1...
31197,31198,"2010-06-12 06:54:06,586514",192.168.56.1,75.127.97.72,TCP,66,52999 > 80 [ACK] Seq=1 Ack=1 Win=29696 Len=0...
31198,31199,"2010-06-12 06:54:06,586762",192.168.56.1,75.127.97.72,TCP,74,53000 > 80 [SYN] Seq=0 Win=29200 Len=0 MSS=1...
31200,31201,"2010-06-12 06:54:06,586919",192.168.56.1,75.127.97.72,TCP,66,53000 > 80 [ACK] Seq=1 Ack=1 Win=29696 Len=0...


In [18]:
protocol=dest[dest.Protocol=="HTTP"]
protocol

,No.,Time,Source,Destination,Protocol,Length,Info
31176,31177,"2010-06-12 06:54:06,348044",192.168.56.1,75.127.97.72,HTTP,710,GET /drupal7/ HTTP/1.1
31192,31193,"2010-06-12 06:54:06,585868",192.168.56.1,75.127.97.72,HTTP,749,GET /drupal7/sites/default/files/styles/medium...
31203,31204,"2010-06-12 06:54:06,586953",192.168.56.1,75.127.97.72,HTTP,734,GET /drupal7/sites/default/files/styles/medium...
31206,31207,"2010-06-12 06:54:06,587263",192.168.56.1,75.127.97.72,HTTP,733,GET /drupal7/sites/default/files/styles/medium...
31208,31209,"2010-06-12 06:54:06,587412",192.168.56.1,75.127.97.72,HTTP,684,GET /drupal7/misc/feed.png HTTP/1.1
31210,31211,"2010-06-12 06:54:06,587497",192.168.56.1,75.127.97.72,HTTP,747,GET /drupal7/sites/default/files/styles/medium...
31335,31336,"2010-06-12 06:54:07,851412",192.168.56.1,75.127.97.72,HTTP,710,GET /drupal7/?q=node/3 HTTP/1.1
31370,31371,"2010-06-12 06:54:07,963788",192.168.56.1,75.127.97.72,HTTP,710,GET /drupal7/modules/filter/filter.css?mpmyqx ...
31374,31375,"2010-06-12 06:54:07,966806",192.168.56.1,75.127.97.72,HTTP,682,GET /drupal7/misc/form.js?v=7.12 HTTP/1.1
31376,31377,"2010-06-12 06:54:07,966823",192.168.56.1,75.127.97.72,HTTP,686,GET /drupal7/misc/textarea.js?v=7.12 HTTP/1.1


In [32]:
dest.Protocol.value_counts()
dest.Info

31173    52998  >  80 [SYN] Seq=0 Win=29200 Len=0 MSS=1...
31175    52998  >  80 [ACK] Seq=1 Ack=1 Win=29696 Len=0...
31176                              GET /drupal7/ HTTP/1.1 
31184    52998  >  80 [ACK] Seq=645 Ack=4345 Win=37888 ...
31186    52998  >  80 [ACK] Seq=645 Ack=4444 Win=37888 ...
31192    GET /drupal7/sites/default/files/styles/medium...
31194    52999  >  80 [SYN] Seq=0 Win=29200 Len=0 MSS=1...
31197    52999  >  80 [ACK] Seq=1 Ack=1 Win=29696 Len=0...
31198    53000  >  80 [SYN] Seq=0 Win=29200 Len=0 MSS=1...
31200    53000  >  80 [ACK] Seq=1 Ack=1 Win=29696 Len=0...
31201    53001  >  80 [SYN] Seq=0 Win=29200 Len=0 MSS=1...
31203    GET /drupal7/sites/default/files/styles/medium...
31205    53001  >  80 [ACK] Seq=1 Ack=1 Win=29696 Len=0...
31206    GET /drupal7/sites/default/files/styles/medium...
31208                 GET /drupal7/misc/feed.png HTTP/1.1 
31210    GET /drupal7/sites/default/files/styles/medium...
31212    53001  >  80 [ACK] Seq=619 Ack=190 Win=30720 L.

In [46]:
trace1=go.Scatter(x=dest['Time'],y=dest['Length'],mode='markers')
d=dest.query('(Length>300)&(Length<600)')
trace2=go.Scatter(x=d['Time'],y=d['Length'], mode='lines')
data=[trace1, trace2]
py.iplot(data)

In [54]:
df['Length']

0        1514
1        1514
2          60
3        1195
4         140
5         925
6         245
7          60
8          62
9          60
10        112
11       1211
12        149
13         60
14         60
15       1514
16        672
17         60
18        143
19         60
20         60
21        610
22         60
23         60
24         60
25         62
26       1514
27       1514
28         60
29       1514
         ... 
58102      70
58103      70
58104      60
58105      63
58106      63
58107      60
58108      72
58109      72
58110      60
58111      60
58112      60
58113      60
58114      60
58115      60
58116      60
58117      60
58118      60
58119      60
58120      60
58121      60
58122      60
58123      60
58124      60
58125      60
58126      60
58127      60
58128      60
58129      60
58130      60
58131      60
Name: Length, Length: 58132, dtype: int64